In [ ]:
from google.colab import drive
import os
import re
drive.mount('/content/drive')

## Crop das images e respectivas segmentações
Todas as 18 imagens(15 de treino + 3 de teste) de 1200x1200 dadas pelo parceiro serão cropadas em 36 imagens de 200x200, assim como suas respectivas masks, e serão armazenadas na pasta 'cropped_images' na pasta de teste e na de treino no drive do grupo

### Treino

In [ ]:
train_tifs_path = '/content/drive/Shared drives/Grupo T de Tech/Data/dataset_inteli/tci_tifs/'
train_masks_path = '/content/drive/Shared drives/Grupo T de Tech/Data/dataset_inteli/masks/'

In [ ]:
from PIL import Image

ordered_masks = sorted(os.listdir(train_masks_path))
ordered_tifs = sorted(os.listdir(train_tifs_path))

In [ ]:
for i in range(len(ordered_masks)):
  count = 0
  mask = Image.open('/content/drive/Shared drives/Grupo T de Tech/Data/dataset_inteli/masks/' + ordered_masks[i])
  tif = Image.open('/content/drive/Shared drives/Grupo T de Tech/Data/dataset_inteli/tci_tifs/' + ordered_tifs[i])
  image_name = re.findall(r'\d+', ordered_masks[i])
  os.mkdir('/content/drive/Shared drives/Grupo T de Tech/Data/dataset_inteli/cropped_images/{}'.format((''.join(image_name))))
  for row in range(6):
    for col in range(6):
      os.mkdir('/content/drive/Shared drives/Grupo T de Tech/Data/dataset_inteli/cropped_images/{}/{}'.format((''.join(image_name)), (count)))
      left = (col)*200
      top = (row)*200
      right = left + 200
      bottom = top + 200

      cropped_mask = mask.crop((left, top, right, bottom))
      cropped_tif = tif.crop((left, top, right, bottom))
      cropped_mask.save(os.path.join('/content/drive/Shared drives/Grupo T de Tech/Data/dataset_inteli/cropped_images/{}/{}'.format((''.join(image_name)), (count)), 'mask.png'))
      cropped_tif.save(os.path.join('/content/drive/Shared drives/Grupo T de Tech/Data/dataset_inteli/cropped_images/{}/{}'.format((''.join(image_name)), (count)), 'image.tif'))
      count += 1

### Teste

In [ ]:
test_tifs_path = '/content/drive/Shared drives/Grupo T de Tech/Data/dataset_inteli_test/tci_tifs/'
test_masks_path = '/content/drive/Shared drives/Grupo T de Tech/Data/dataset_inteli_test/masks/'

In [ ]:
from PIL import Image

ordered_masks = sorted(os.listdir(test_masks_path))
ordered_tifs = sorted(os.listdir(test_tifs_path))

In [ ]:
for i in range(len(ordered_masks)):
  count = 0
  mask = Image.open('/content/drive/Shared drives/Grupo T de Tech/Data/dataset_inteli_test/masks/' + ordered_masks[i])
  tif = Image.open('/content/drive/Shared drives/Grupo T de Tech/Data/dataset_inteli_test/tci_tifs/' + ordered_tifs[i])
  image_name = re.findall(r'\d+', ordered_masks[i])
  os.mkdir('/content/drive/Shared drives/Grupo T de Tech/Data/dataset_inteli_test/cropped_images/{}'.format((''.join(image_name))))
  for row in range(6):
    for col in range(6):
      os.mkdir('/content/drive/Shared drives/Grupo T de Tech/Data/dataset_inteli_test/cropped_images/{}/{}'.format((''.join(image_name)), (count)))
      left = (col)*200
      top = (row)*200
      right = left + 200
      bottom = top + 200

      cropped_mask = mask.crop((left, top, right, bottom))
      cropped_tif = tif.crop((left, top, right, bottom))
      cropped_mask.save(os.path.join('/content/drive/Shared drives/Grupo T de Tech/Data/dataset_inteli_test/cropped_images/{}/{}'.format((''.join(image_name)), (count)), 'mask.png'))
      cropped_tif.save(os.path.join('/content/drive/Shared drives/Grupo T de Tech/Data/dataset_inteli_test/cropped_images/{}/{}'.format((''.join(image_name)), (count)), 'image.tif'))
      count += 1

## Carregando as imagens e aplicando kmeans e sharpen

In [ ]:
os.listdir('/content/drive/Shared drives/Grupo T de Tech/Data/dataset_inteli/cropped_images/575/0')

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import tqdm
from glob import glob

class KMeansImageProcessingPipeline:
	def __init__(self, base_dir):
		self.base_dir = base_dir

	def read_and_process_image(self, path):
		img = cv2.imread(path)
		img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
		return img

	def kmeans_image(self, img, k, attempts=10):
		# Redimensionar a imagem para um vetor 1D
		vectorized_img = img.reshape((-1,3))
		vectorized_img = np.float32(vectorized_img)

		# Definir critérios para o algoritmo K-means
		criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)

		# Executar o algoritmo K-means
		ret, label, center = cv2.kmeans(vectorized_img, k, None, criteria, attempts, cv2.KMEANS_PP_CENTERS)
		center = np.uint8(center)

		# Recriar a imagem a partir dos clusters encontrados
		res = center[label.flatten()]
		result_image = res.reshape((img.shape))

		return result_image

	def sharpen_image(self, img):
		# Cria um kernel para afiar a imagem
		kernel = np.array([[-1,-1,-1],
											[-1,9,-1],
											[-1,-1,-1]])

		# Aplica o kernel na imagem usando filter2D
		sharpened_image = cv2.filter2D(img, -1, kernel)

		return sharpened_image

	def process_all_images(self, k, attempts=10):
		img = self.read_and_process_image(self.base_dir)

		# Aplica o algoritmo de k-means na imagem
		kmeans_img = self.kmeans_image(img, k, attempts)

		# Afia a imagem resultante
		sharpened_img = self.sharpen_image(kmeans_img)

		return sharpened_img

	def analyze_kmeans_inertia(self, k_range):
		sample_image_path = os.path.join(self.base_dir, os.listdir(self.base_dir)[0])
		img = self.read_and_process_image(sample_image_path)
		vectorized_img = img.reshape((-1, 3))
		vectorized_img = np.float32(vectorized_img)

		inertias = []
		for k in k_range:
				kmeans = KMeans(n_clusters=k, random_state=42)
				kmeans.fit(vectorized_img)
				inertias.append(kmeans.inertia_)

		plt.figure(figsize=[20, 5])
		plt.subplot(1, 2, 1)
		plt.plot(k_range, inertias, "-o")
		plt.xlabel("$k$", fontsize=14)
		plt.ylabel("Inertia", fontsize=14)
		plt.grid(True)

		plt.subplot(1, 2, 2)
		plt.plot(k_range[:-1], np.diff(inertias), "-o")
		plt.xlabel("$k$", fontsize=14)
		plt.ylabel("Change in inertia", fontsize=14)
		plt.grid(True)
		plt.show()

# Usage
# base_dir = '../../data/dataset_inteli/tci_pngs'
processed_images = []
for caminho in glob('/content/drive/Shared drives/Grupo T de Tech/Data/dataset_inteli/cropped_images/*/*/*.tif'):
	print(caminho)
	pipeline = KMeansImageProcessingPipeline(caminho)
	processed_images.append(pipeline.process_all_images(5))  # Change k as needed
# pipeline.analyze_kmeans_inertia(range(4, 16))


In [ ]:
for img in processed_images[:20]:
	plt.imshow(img)
	plt.show()

## Aplicando Data Augmentation

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

class ImageProcessingPipeline:
	def __init__(self, images):
		self.images = images

	@staticmethod
	def normalize_image(image):
		return image / 255.0

	@staticmethod
	def resize_image(image, target_size=(1200, 1200)):
		return cv2.resize(image, target_size)

	def apply_clahe(self, image):
		clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
		return clahe.apply(image.astype(np.uint8))

	@staticmethod
	def crop_image(image, crop_size=(200, 200)):
		crops = []

		for i in range(0, image.shape[0], crop_size[0]):

			for j in range(0, image.shape[1], crop_size[1]):
					crop = image[i:i+crop_size[0], j:j+crop_size[1]]

					if crop.shape[0] == crop_size[0] and crop.shape[1] == crop_size[1]:
						crops.append(crop)

		return crops

	def augment_images(self, image):
		aug_images = []

		for angle in [0, 90, 180, 270]:
			rotated = self.rotate_image(image, angle)
			aug_images.append(rotated)
			aug_images.append(cv2.flip(rotated, 1))

		return aug_images


	@staticmethod
	def rotate_image(image, angle):
		(h, w) = image.shape[:2]
		center = (w // 2, h // 2)

		M = cv2.getRotationMatrix2D(center, angle, 1.0)

		return cv2.warpAffine(image, M, (w, h))

	def process_images(self):
		processed_images = []

		for img in self.images:
			norm_img = ImageProcessingPipeline.normalize_image(img)
			resized_img = ImageProcessingPipeline.resize_image(norm_img)

			cropped_images = self.crop_image(resized_img)

			for crop in cropped_images:
				augmented_imgs = self.augment_images(crop)
				processed_images.extend(augmented_imgs)

		return processed_images

	def show_image(self, image):
		plt.imshow(image, cmap='gray')
		plt.axis('off')
		plt.show()

In [ ]:
pipeline = ImageProcessingPipeline(processed_images)
processed_images_pipeline = pipeline.process_images()

# Mostrar algumas das imagens processadas
for img in processed_images_pipeline:  # Mostra as primeiras 8 imagens processadas
    pipeline.show_image(img)

len(processed_images_pipeline)